In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pymc3 as pm
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import arviz as az 
%matplotlib inline
pd.options.display.max_columns = None

In [ ]:
df_game = pd.read_csv('/kaggle/input/mobile-games-ab-testing/cookie_cats.csv')

# Avaliação sobre a melhor opção entre os portões utilizando a lib Pymc3

In [ ]:
#Avaliando as colunas e valores nulos
df_game.info()

### Não observamos valores nulos no Dataset.

In [ ]:
#Estatística básicas 
df_game.describe()

In [ ]:
sns.boxplot(df_game['sum_gamerounds'])

### Observa-se um valor máximo na variável sum_gamerounds muito acima da média. Vamos tratar os outliers

In [ ]:
#removendo outliers
Q1 = df_game['sum_gamerounds'].quantile(0.25)
Q3 = df_game['sum_gamerounds'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
df_game.shape[0] - \
df_game[(df_game['sum_gamerounds'] < (Q1 - 1.5 * IQR)) | (df_game['sum_gamerounds'] > (Q3 + 1.5 * IQR))].shape[0]

Utilizando a abordagem de IQR removemos um volume muito alto de registros. Vamos testar a abordagem por Z-score

In [ ]:
z = np.abs(stats.zscore(df_game['sum_gamerounds']))
print(z)

In [ ]:
df_outdropped = df_game[(z < 2)]

In [ ]:
df_game.shape[0] - df_outdropped.shape[0]

In [ ]:
sns.boxplot(df_outdropped['sum_gamerounds'])

In [ ]:
df_outdropped.describe()

### Utilizando Z-score, chegamos a uma distribuição mais coerente, sem perder muitos dados.

In [ ]:
df_game = df_outdropped.copy()

In [ ]:
#analisando os grupos por versão do gate (30 ou 40)
df_game.groupby('version').count()[['userid']]

### Observa-se que a base foi dividida em proporções praticamente iguais.

In [ ]:
#Análise da distribuição de gamerounds por gate
gate_30 = df_game.query("version == 'gate_30'")['sum_gamerounds'].values
gate_40 = df_game.query("version == 'gate_40'")['sum_gamerounds'].values

In [ ]:
plt.figure(figsize=(12,5))
sns.distplot(gate_30)
sns.distplot(gate_40)

In [ ]:
print(f'Média Gate 30: {gate_30.mean()}')
print(f'Média Gate 40: {gate_40.mean()}')

### Em termos da média de Gamerounds observamos uma pequena diferença a favor do Gate 30.

# Análise da métrica retenção de 1 dia para os Gates 30 e 40

In [ ]:
g30_1 = df_game.query("version == 'gate_30'")['retention_1'].values.astype(int)
g40_1 = df_game.query("version == 'gate_40'")['retention_1'].values.astype(int)
print(f'Retenção média Gate 30: {100*g30_1.mean().round(3)}')
print(f'Retençao média Gate 40: {100*g40_1.mean().round(3)}')
print(f'Diferença da média entre Gate 30 e Gate 40: {(g30_1.mean() - g40_1.mean()).round(3)*100}')

### Aqui observamos que para a retenção de 01 dia o Gate 30, a exemplo da média geral de Gamerounds, apresenta um valor de 0.6 pp em relação ao 40. Vamos rodar o modelo do Pymc3 para validar a significância desta diferença.

## Informações sobre as distribuições do modelo Bayesiano

### * Priori: Uniforme com intervalo entre [0,1]


### * Likelihood: Bernoulli


In [ ]:
with pm.Model() as model_30:
    p_30 = pm.Uniform('p_30',lower=0, upper=1)
    
    obs_30 = pm.Bernoulli('obs_30', p_30, observed=g30_1)
    
    step = pm.Metropolis()
    trace = pm.sample(2000, step = step)
    burned_trace_30 = trace[1000:]

In [ ]:
with pm.Model() as model_40:
    p_40 = pm.Uniform('p_40',lower=0, upper=1)
    
    obs_40 = pm.Bernoulli('obs_40', p_40, observed=g40_1)
    
    step = pm.Metropolis()
    trace = pm.sample(2000, step = step)
    burned_trace_40 = trace[1000:]

In [ ]:
#Observando a média das duas distribuições com os valores posteriori gerados
print(f'Média posteriori Gate 30: {burned_trace_30["p_30"].mean().round(3)}')
print(f'Média posteriori Gate 40: {burned_trace_40["p_40"].mean().round(3)}')

In [ ]:
plt.figure(figsize=(12.5, 4))
plt.hist(burned_trace_30["p_30"], bins=40, label='Posterior P_30', density=True)
plt.hist(burned_trace_40["p_40"], bins=40, label='Posterior P_40', density=True)
plt.xlabel('Valores')
plt.ylabel('Densidade')
plt.title("Distribuição posterior para a retenção de 1 dia $30$ and group $40$")
plt.legend()
plt.show()

### Na distribuição posterior, temos valores maiores para o Gate 30. Vamos aprofundar um pouco mais observando a diferença absoluta entre as retenções

In [ ]:
diff=burned_trace_30["p_30"]-burned_trace_40["p_40"]
plt.figure(figsize=(12.5, 4))
plt.hist(diff, bins=40, density=True)
plt.vlines(0, 0, 120, linestyle='--', color='red')
plt.title('Distribuição posterior para a diferença entre as duas médias')
plt.show()

### Pelo gráfico acima, observamos que boa parte da distribuição da diferença absoluta está acima de 0, o que indica que na maioria dos dados a conversão de 1 dia para o Gate 30 é maior do que o Gate 40.

In [ ]:
#Probabilidade da diferença ser maior do que 0
100*len(diff[diff>0.0])*1.0/len(diff)

In [ ]:
#Avaliando a diferença relativa
rel_diff=100*(burned_trace_30["p_30"]-burned_trace_40["p_40"])/burned_trace_40["p_40"]
plt.figure(figsize=(12.5, 4))
plt.hist(rel_diff, bins=40, density=True)
plt.vlines(0, 0, 0.7, linestyle='--', color='red')
plt.title('Diferença relativa entre as duas retenções médias')
plt.xlabel("Porcentagem")
plt.show()

In [ ]:
100*len(rel_diff[rel_diff>0.0])*1.0/len(rel_diff)

# Conclusão Retenção de 1 dia

### Pelas observações acima, temos uma confiança de 96% que a retenção para o Gate 30 é maior do que o 40.

# Análise da Retenção de 7 Dias

In [ ]:
g30_7 = df_game.query("version == 'gate_30'")['retention_7'].values.astype(int)
g40_7 = df_game.query("version == 'gate_40'")['retention_7'].values.astype(int)
print(f'Retenção média Gate 30: {100*g30_7.mean().round(3)}')
print(f'Retençao média Gate 40: {100*g40_7.mean().round(3)}')
print(f'Diferença da média entre Gate 30 e Gate 40: {(g30_7.mean() - g40_7.mean()).round(3)*100}')

In [ ]:
with pm.Model() as model_7_30_v2:
    p_7_30 = pm.Uniform('p_7_30',lower=0, upper=1)
    p_7_40 = pm.Uniform('p_7_40',lower=0, upper=1)
    
    obs_7_30 = pm.Bernoulli('obs_7_30', p_7_30, observed=g30_7)
    obs_7_40 = pm.Bernoulli('obs_7_40', p_7_40, observed=g40_7)
    
    diff = pm.Deterministic('diff', p_7_30 - p_7_40)
    rel_diff = pm.Deterministic('rel_diff', 100*(p_7_30 - p_7_40)/p_7_40)
    
    step = pm.Metropolis()
    trace = pm.sample(2000, step = step)
    burned_trace_7 = trace[1000:]

In [ ]:
#Observando a média das duas distribuições com os valores posteriori gerados
print(f'Média posteriori Gate 30: {burned_trace_7["p_7_30"].mean().round(3)}')
print(f'Média posteriori Gate 40: {burned_trace_7["p_7_40"].mean().round(3)}')

In [ ]:
plt.figure(figsize=(12.5, 4))
plt.hist(burned_trace_7["p_7_30"], bins=40, label='Posterior P_30', density=True)
plt.hist(burned_trace_7["p_7_40"], bins=40, label='Posterior P_40', density=True)
plt.xlabel('Valores')
plt.ylabel('Densidade')
plt.title("Distribuição posterior para a retenção de 7 dia $30$ and group $40$")
plt.legend()
plt.show()

In [ ]:
diff=burned_trace_7["p_7_30"]-burned_trace_7["p_7_40"]
plt.figure(figsize=(12.5, 4))
plt.hist(diff, bins=40, density=True)
plt.vlines(0, 0, 120, linestyle='--', color='red')
plt.title('Distribuição posterior para a diferença entre as duas médias na retenção de 7 dias')
plt.show()

In [ ]:
#Probabilidade da diferença ser maior do que 0
100*len(diff[diff>0.0])*1.0/len(diff)

In [ ]:
#Avaliando a diferença relativa
rel_diff=100*(burned_trace_7["p_7_30"]-burned_trace_7["p_7_40"])/burned_trace_7["p_7_40"]
plt.figure(figsize=(12.5, 4))
plt.hist(rel_diff, bins=40, density=True)
plt.vlines(0, 0, 0.7, linestyle='--', color='red')
plt.title('Diferença relativa entre as duas retenções médias')
plt.xlabel("Porcentagem")
plt.show()

In [ ]:
100*len(rel_diff[rel_diff>0.0])*1.0/len(rel_diff)

# Conclusão Retenção de 7 dia

Pelas observações acima, temos uma confiança de 99,95% que a retenção para o Gate 30 é maior do que o 40 para 7 dias.
Como conclusão final observamos que colocar o portão no Level 30 trouxe maior retenção tanto para as métricas de 1 e 7 dias. Para a tomada de decisão em um problema real outras variáveis precisariam ser analisadas, porém, caso a decisão precisasse ser tomada levando em conta somente as estudadas, a manutenção do Gate 30 seria a mais adequada. 
